Based on https://bambinos.github.io/bambi/notebooks/shooter_crossed_random_ANOVA.html

In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from data_utils import download_single_csv
from pathlib import Path

In [ ]:
az.style.use("arviz-darkgrid")
SEED = 1234

In [ ]:
path = Path("./data")
name = "shooter.csv"
download_single_csv(path, name)

In [ ]:
shooter = pd.read_csv("data/shooter.csv", na_values=".")
shooter.head(10)

In [ ]:
shooter["rate"] = 1000.0 / shooter["time"]

In [ ]:
plt.hist(shooter["rate"].dropna());

In [ ]:
subj_model = bmb.Model(
    "rate ~ S(race) * S(object) + (S(race) * S(object) | subject)", shooter, dropna=True
)
subj_fitted = subj_model.fit(random_seed=SEED)

In [ ]:
subj_model.plot_priors();

In [ ]:
az.plot_trace(subj_fitted);

In [ ]:
az.summary(subj_fitted)

In [ ]:
stim_model = bmb.Model(
    "rate ~ S(race) * S(object) + (S(race) * S(object) | subject) + (S(object) | target)",
    shooter,
    dropna=True,
)
stim_fitted = stim_model.fit(random_seed=SEED)

In [ ]:
az.plot_trace(stim_fitted);

In [ ]:
az.summary(stim_fitted)

In [ ]:
shooter["shoot_or_not"] = shooter["response"].astype(str)

# armed targets
new_values = {"correct": "shoot", "incorrect": "dontshoot", "timeout": np.nan}
shooter.loc[shooter["object"] == "gun", "shoot_or_not"] = (
    shooter.loc[shooter["object"] == "gun", "response"].astype(str).replace(new_values)
)

# unarmed targets
new_values = {"correct": "dontshoot", "incorrect": "shoot", "timeout": np.nan}
shooter.loc[shooter["object"] == "nogun", "shoot_or_not"] = (
    shooter.loc[shooter["object"] == "nogun", "response"]
    .astype(str)
    .replace(new_values)
)

# view result
shooter.head(20)

In [ ]:
stim_response_model = bmb.Model(
    "shoot_or_not[shoot] ~ S(race)*S(object) + (S(race)*S(object) | subject) + (S(object) | target)",
    shooter,
    family="bernoulli",
    dropna=True,
)

# Note we increased target_accept from default 0.8 to 0.9 because there were divergences
stim_response_fitted = stim_response_model.fit(
    draws=2000, target_accept=0.9, random_seed=SEED
)

In [ ]:
az.plot_trace(stim_response_fitted);

In [ ]:
az.summary(stim_response_fitted)

In [ ]:
(stim_response_fitted.posterior["S(race)"] < 0).mean()